## Generative AI PDF information exctractor
This is my Master thesis project which results in an AI app to make a LLM from Open AI capable of answer specific questions about a topic. The model is able to access external data that hasn't seen during training, understand a user's question, exctract the information it needs to answer the user and return a comprehensive answer to the question formulated by the user. In the image below, there's a schema of the pipeline from user's query to final output:

In [1]:
# pip insatall langchain
# pip instal pypdf
# pip install tiktoken
# pip install faiss-cpu

In [2]:
pdf_path = "C:/Users/Nacho/Documents/MASTER/TFM/1-Pruebas_piloto/External data/BOE-034_Codigo_Civil_y_legislacion_complementaria.pdf"

# Load pdf with external info not seen during training of the LLM
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader(pdf_path)
pages = loader.load_and_split()

ImportError: pypdf package not found, please install it with `pip install pypdf`

In [ ]:
# Generate vector space represetnation with words from the external data
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [ ]:
# Load embeddings in vector database
from langchain.vectorstores import FAISS
db = FAISS.from_documents(pages, embeddings)

# q = "What is Link's traditional outfit color?"
# db.similarity_search(q)[0]

In [ ]:
# Use information retrieval from embedding for answer
from langchain.chains import RetrievalQA
from langchain import OpenAI
llm = OpenAI()
chain = RetrievalQA.from_llm(llm=llm, retriever=db.as_retriever())
q = "What is Link's traditional outfit color?"
chain(q, return_only_outputs=True)